In [1]:
import numpy as np
import pandas as pd
import F1Optimizer as f1Opt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb

In [2]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')




In [3]:

target_train = df_train['y'].values
id_test = df_test['id'].values

In [ ]:
for col in df_test.columns:
    if (len(df_test[col].unique()) < 100) and (len(df_test[col].unique()) == len(df_train[col].unique())):
        df_train = pd.get_dummies(df_train, columns=[col])
        df_test = pd.get_dummies(df_test, columns=[col])

In [ ]:
for col in df_test.columns:
    
    if col not in df_train.columns:
        df_train[col] = 0
        

In [36]:
train = np.array(df_train.drop(['y','id'], axis = 1))
test = np.array(df_test.drop(['id'], axis = 1))

xgb_preds = []

In [37]:
K = 5
kf = KFold(n_splits = K, random_state = 3228, shuffle = True)
cutPoint = 0.36

In [38]:
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
def xgb_f1(y, t, threshold=0.5):
    t = t.get_label()
    y_bin = (y > threshold).astype(int) # works for both type(y) == <class 'numpy.ndarray'> and type(y) == <class 'pandas.core.series.Series'>
    return 'f1',f1_score(t,y_bin)
err = 0
errAuc = 0
optG = 0
ff = 1
for train_index, test_index in kf.split(train, target_train):
    train_X, valid_X = train[train_index], train[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]

    # params configuration also from the1owl's kernel
    # https://www.kaggle.com/the1owl/forza-baseline
    xgb_params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}
    #xgb_params = {'eta': 0.4454, 'max_depth': 3, 'subsample': 0.4, 'colsample_bytree': 0.4, 'objective': 'binary:logistic', 'min_child_weight': 0.01004, 'eval_metric': 'auc', 'seed': 99, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000, watchlist, maximize=True, verbose_eval=50, early_stopping_rounds=100)
                        
    xgb_pred = model.predict(d_test)
    p = model.predict(d_valid)

    errAuc += log_loss(valid_y, p)
    pNone = (1.0 - p).prod()
    #opt = f1Opt.F1Optimizer.maximize_expectation(p, pNone)
    #opt = 1 - opt[2]
    #optG += opt
    #opt = optG / ff # test here
    ff += 1
    p[p > cutPoint] = 1
    p[p != 1] = 0
    err += f1_score(valid_y, p)
    print(err/ff)
    xgb_preds.append(list(xgb_pred))

[20:31:34] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.85281	valid-auc:0.83511
[50]	train-auc:0.87725	valid-auc:0.85379
[100]	train-auc:0.88476	valid-auc:0.86054
[150]	train-auc:0.89583	valid-auc:0.86989
[200]	train-auc:0.90640	valid-auc:0.87529
[250]	train-auc:0.91472	valid-auc:0.87910
[300]	train-auc:0.92146	valid-auc:0.88156
[350]	train-auc:0.92702	valid-auc:0.88280
[400]	train-auc:0.93232	valid-auc:0.88462
[450]	train-auc:0.93711	valid-auc:0.88527
[500]	train-auc:0.94129	valid-auc:0.88595
[550]	train-auc:0.94524	valid-auc:0.88637
[600]	train-auc:0.94848	valid-auc:0.88659
[650]	train-auc:0.95139	valid-auc:0.88714
[700]	train-auc:0.95438	valid-auc:0.88766
[750]	train-auc:0.95729	valid-auc:0.88820
[80

In [45]:
print(err/5) #print(err/4) #6711266 print(err/5) #print(err/4) #6711266
print(errAuc/5) #print(err/4) #6711266 print(err/5) #print(err/4) #6711266
print(optG) #print(err/4) #6711266 print(err/5) #print(err/4) #6711266
#cutPoint = (optG/5) + .025
# 0.6618374999045851
# 0.3165819770567079
#print(cutPoint) #print(err/)

0.6711266450985601
0.3081443192997369
0


In [46]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

output = pd.DataFrame({'id': id_test, 'predicted': preds})

output['predicted'][output['predicted'] > cutPoint] = 1
output['predicted'][output['predicted'] != 1] = 0
output['predicted'] = output['predicted'].astype(int)
output.to_csv("{}-foldCV_avg_sub_{}_auc_{}_Alldummy_strat.csv".format(err/5, cutPoint, errAuc/5), index=False)   

<ipython-input-46-9beb95b77828>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predicted'][output['predicted'] > cutPoint] = 1
<ipython-input-46-9beb95b77828>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predicted'][output['predicted'] != 1] = 0


In [41]:
sub = pd.read_csv("./5-foldCV_avg_sub_36_dummy.csv")

In [42]:
sub["v"] = output["predicted"]

In [43]:
sub["v2"] = sub["v"] - sub["predicted"]

In [47]:
sub.v2.describe()

count    21183.000000
mean        -0.000330
std          0.094459
min         -1.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: v2, dtype: float64